# Assemblage liste de données

## Ajout d'un colonne & Fusion des fichier

In [105]:
import os
import pandas as pd

In [106]:
dossier = "CSV"
fichiers_a_inclure = ["annapurna-I.csv", "broad-peak.csv", "cho-oyu.csv", "dhaulagiri-I.csv", "everest.csv", "gasherbrum-I.csv", "gasherbrum-II.csv", "k2.csv", "kangchenjunga.csv", "lhotse.csv", "makalu.csv", "manaslu.csv", "nanga-parbat.csv", "shishapangma.csv"]

In [107]:
# Test a enlever
noms_fichiers_formatés = []

for fichier in os.listdir(dossier):
    if fichier.endswith(".csv"):
        # Suppression de l'extension
        nom_sans_extension = fichier.replace('.csv', '')
        # Rajout d'une majuscule au début du nom
        nom_formaté = nom_sans_extension[0].upper() + nom_sans_extension[1:]
        noms_fichiers_formatés.append(nom_formaté)
noms_fichiers_formatés

['Annapurna-I',
 'Broad-peak',
 'Cho-oyu',
 'Dhaulagiri-I',
 'Everest',
 'Gasherbrum-I',
 'Gasherbrum-II',
 'K2',
 'Kangchenjunga',
 'Lhotse',
 'Makalu',
 'Manaslu',
 'Mountain_ranges',
 'Nanga-parbat',
 'Shishapangma']

In [108]:
donnees_totales = []

In [109]:
for fichier in fichiers_a_inclure:
    chemin_fichier = os.path.join(dossier, fichier)
    # Lecture du fichier CSV
    df = pd.read_csv(chemin_fichier)
    # Ajout de la colonne "file" avec le nom du fichier (sans extension .csv)
    nom_sans_extension = fichier.replace('.csv', '')
    df['Montagne'] = nom_sans_extension[0].upper() + nom_sans_extension[1:]
    # Ajout du DataFrame à la liste
    donnees_totales.append(df)
 

In [110]:
# Concaténation de tous les DataFrames en un seul
donnees_combinees = pd.concat(donnees_totales, ignore_index=True)
# Enregistrement des données combinées dans un fichier CSV
donnees_combinees.to_csv('Donnees_Combinees.csv', index=False)

## Déplacement de colonne

In [111]:
# Chemin vers le fichier CSV d'entrée et de sortie
chemin_fichier_entree = "Donnees_Combinees.csv"
chemin_fichier_sortie = "Donnees_Combinees.csv"

In [112]:
# Lire le fichier CSV avec pandas
donnees = pd.read_csv(chemin_fichier_entree)

In [113]:
# Réorganiser les colonnes, en mettant 'Montagne' devant 'Date'
colonnes = ['Montagne', 'Date', 'Name', 'Nationality', 'Cause of death']

In [114]:
# Réorganiser les colonnes du DataFrame
donnees = donnees[colonnes]

In [115]:
# Écrire le DataFrame modifié dans un nouveau fichier CSV
donnees.to_csv(chemin_fichier_sortie, index=False)

## Netoyage des donné génante et inutile

In [116]:
# Chemin vers le fichier CSV d'entrée et de sortie
chemin_fichier_entree = "Donnees_Combinees.csv"
chemin_fichier_sortie = "Donnees_Combinees.csv"

# Lire le fichier CSV avec pandas
donnees = pd.read_csv(chemin_fichier_entree)

# Appliquer une expression régulière pour enlever le contenu entre crochets `[ ]` en tenant compte de l'espace éventuel avant `[`
donnees = donnees.replace(to_replace=r'(\s*)\[(.*?)\]', value='', regex=True)

# Écrire le DataFrame modifié dans un nouveau fichier CSV
donnees.to_csv(chemin_fichier_sortie, index=False)

## Date changement de format

In [117]:
from datetime import datetime

In [118]:
# Fonction pour convertir la date en format jour/mois/année en nombre
def convertir_date(date_str):
    # Liste des formats possibles de date
    formats = ['%d %B %Y', '%B %d, %Y', '%Y, %dth %B', '%B %Y', '%Y']

    # Essayer de convertir la date avec différents formats
    for fmt in formats:
        try:
            date_obj = datetime.strptime(date_str, fmt)
            # Si la date ne contient pas de jour, retourner seulement le mois et l'année
            if '%d' not in fmt:
                return date_obj.strftime('%m/%Y')
            else:
                return date_obj.strftime('%d/%m/%Y')
        except ValueError:
            pass
    # Si aucun format ne correspond, retourner la date telle quelle
    return date_str

In [119]:
import dateparser

def convertir_date(date_str):
    # Essayer de parser la date avec dateparser
    parsed_date = dateparser.parse(date_str, languages=['en'])
    if parsed_date:
        # Si la date n'inclut pas de jour précis, retourner seulement le mois et l'année
        if parsed_date.day == 1:
            return parsed_date.strftime('%m/%Y')
        else:
            return parsed_date.strftime('%d/%m/%Y')

    # Si la date ne peut pas être analysée, retourner la date telle quelle
    return date_str


In [120]:
# Chemin vers le fichier CSV d'entrée et de sortie
chemin_fichier_entree = "Donnees_Combinees.csv"
chemin_fichier_sortie = "Donnees_Modifiees.csv"

# Lire le fichier CSV avec pandas
donnees = pd.read_csv(chemin_fichier_entree)

donnees['Date'] = donnees['Date'].replace(to_replace="Summer 1990", value="07/1990")
donnees['Date'] = donnees['Date'].replace(to_replace="Winter 1999", value="01/1999")
donnees['Date'] = donnees['Date'].replace(to_replace="mid July 1982", value="15/07/1982")
donnees['Date'] = donnees['Date'].replace(to_replace="late June 1975", value="28/06/1975")
donnees['Date'] = donnees['Date'].replace(to_replace="19 May 1996 or May 1997", value="19/05/1996")

# Appliquer la fonction de conversion sur la colonne 'Date'
donnees['Date'] = donnees['Date'].apply(convertir_date)

# Réorganiser les colonnes, en mettant 'Montagne' devant 'Date'
colonnes = ['Montagne', 'Date', 'Name', 'Nationality', 'Cause of death']

# Réorganiser les colonnes du DataFrame
donnees = donnees[colonnes]

# Écrire le DataFrame modifié dans un nouveau fichier CSV
donnees.to_csv(chemin_fichier_sortie, index=False)